# Examples for using PyThermo

## Forward modeling

The most straight-forward application of PyThermo is in forward modeling apatite and zircon (U-Th)/He dates. First, set-up the PyThermo package. You can do this one of two ways. Either install with pip (see this [helpful](https://packaging.python.org/en/latest/tutorials/installing-packages/) guide for using pip if you are unfamiliar) by using this bit of text `pip install pythermo`, or co-locate the folder `/src` with its various files (`__int__.py`,`constants.py`, `crystal.py`, `tT_model.py`, and `tT_path.py`) within the same directory as this notebook. Below I include example code with both approaches.

If you've installed PyThermo, run the bit of code below. If you have not installed PyThermo and would like to run from the source folder, replace the code block below with:
```
import sys
sys.path.append('./src')
import pythermo as pyt
```

In [ ]:
import pythermo as pyt

Now we're ready to demo the forward modeling setup. Import NumPy and Pandas, which you'll need for the data structures used as input for the forward model. 

In [ ]:
import numpy as np
import pandas as pd

We'll create the time-temperature (tT) path array first. This array should be organized in sets of 2 columns, with the first column as time (in millions of years), increasing from 0 (present day) to the oldest time, and the second column as the temperature (in $\degree$ C) that corresponds to each time step. Multiple time-temperature paths can be included, with each individual path represented by two columns. Some paths will have more points than others. If this is the case, then the paths with fewer points should have rows of 0's below their oldest tT point. 

As an example, here's an array with 4 time-temperature paths with variable numbers of points. Running the code cell will display the array so that you have a sense for how it should look.

In [ ]:
#there are several ways to do this, here I'm building by rows (8 entries per row = 4 paths) alternating time, temp
#as an example: the first tT path runs from 20 oC at 0 Ma, to 120 oC at 25 Ma
tT_in_test = np.array([[0,20,0,20,0,20,0,20],[25,120,50,60,75,80,100,140],[0,0,60,40,230,50,0,0],[0,0,270,50,0,0,0,0]])
tT_in_test

Next, construct a DataFrame to hold the grain inputs that will be modeled.

The layout for this DataFrame looks like this:

|          | mineral type   |   age (Ma) |   error (Ma) | size ($\mu$ m) |   U ppm |   Th ppm |   Sm ppm | diffusion model   | annealing model   |
|:---------|:---------------|------:|--------:|-------:|--------:|---------:|---------:|:------------------|:------------------|
| sample 1 | 'apatite'        |    60 |       2 |     60 |    10 |      10 |        10 | 'flowers'           | 'ketcham'           |
| sample 2 | 'apatite'        |    80 |       2 |     60 |    50 |      10 |        10 | 'flowers'           | 'ketcham'           |
| sample 3 | 'apatite'        |    90 |       2 |     60 |    150 |      10 |        10 | 'flowers'          | 'ketcham'           |

A few things:

1. The exact name of each column does not matter (you can call it whatever you want), but the ordering __does matter__. 

2. You can order the inputs however you want, but for plotting purposes, the grains should be organized by increasing eU concentration.

3. There are several ways to plot different curves for grain size. These are explored further below. 

4. Different mineral systems can be run in the same DataFrame, but again, for plotting purposes, you should organize your inputs into separate DataFrames by mineral system (one for apatites, and another for zircons). The currently available inputs for mineral type are 'apatite' and 'zircon'. If you're running zircons, change each entry in that column to 'zircon'.

5. Future versions of this package will allow for comparisons among or between diffusion and annealing models. Currently, each system has only one set of damage annealing and diffusion models. In apatite, 'flowers' corresponds to the damage-diffusivity relationship of Flowers et al. (2009) (https://doi.org/10.1016/j.gca.2009.01.015), and 'ketcham' corresponds to the apatite fission track annealing kinetics of Ketcham et al. (2007) (https://doi.org/10.2138/am.2007.2281). In zircon, 'guenthner' corresponds to the damage-diffusivity relationship of Guenthner et al. (2013) (https://doi.org/10.2475/03.2013.01), and 'guenthner' corresponds to the zircon fission track annealing kinetics as derived in the same publication.

6. The 'age' and 'error' columns are currently placeholders and are __not__ used in the forward model.

7. If you want a straight-forward approach to modeling based on eU, you can enter 0's for the Th and Sm columns. Note that there will be some minor discrepancy here when comparing to measured data as the Th and Sm alpha ejection lengths are different than U.

The code below creates the DataFrame as displayed in this markdown.

In [ ]:
dict = {'mineral type':['apatite', 'apatite', 'apatite'], 'age':[60,80,90],'error':[2,2,2],'size':[60,60,60],'U ppm':[10,50,150],'Th ppm':[10,10,10],'Sm ppm':[10,10,10],'diffusion model':['flowers','flowers','flowers'],'annealing model':['ketcham','ketcham','ketcham']}

grain_in = pd.DataFrame(data=dict,
                        columns = ['mineral type', 'age', 'error','size','U ppm', 'Th ppm', 'Sm ppm', 'diffusion model', 'annealing model'],
                        index = ['sample 1', 'sample 2', 'sample 3'])

grain_in

You may want to compare your forward model results to measured data. To do so, create a three column array with dates (Ma) in the first columns, 2 $\sigma$ error (Ma) in the second column, eU (ppm) in the third column, and grain size ($\mu$ m) in the fourth. Each row represents a single (U-Th)/He grain. Unlike the DataFrame, the ordering here does not matter for plotting purposes. This array is an optional input, as you will see below.

In [ ]:
obs_data = np.array([[62.8,1.2,60,80],[70.1,2.1,20,85],[55.2,0.9,120,65]])
obs_data

We're set to run! Create a tT_model object using your grain_in DataFrame and tT_in_test array. You can also pass the optional keyword variable `temp_precision`. For example: `pyt.tT_model(grain_in, tT_in_test, temp_precision=1)`, which will adjust the maximum allowed spacing between temperature points in the interpolated tT path. Higher numbers results in faster run-times at the cost of increased imprecision on the (U-Th)/He date. The default value (which is used below by not passing the variable) is 5.

In [ ]:
model_test = pyt.tT_model(grain_in,tT_in_test)

The next set of options highlight several ways to model different curves for grain size effects. The first is to call `.forward()` with no additional parameters. This approach will use only the grain sizes input into the DataFrame. If you included only one grain size, you will get only one solid curve per tT path. If you included different grain sizes, you will get three curves per tT path: 1 solid curve for the mean size, and 2 dashed curves corresponding to the $\pm$ 1 $\sigma$ standard deviation.

In [ ]:
model_data = model_test.forward()

A second option is to include a standard deviation as a parameter passed to `.forward()`. This will also yield three curves per tT path

In [ ]:
model_data = model_test.forward(std_grain=30)

A third option is to pass the obs_data array to the model_test object, and then run `.forward()` method. This approach uses the measured data's grain sizes to calculate a mean and $\pm$ 1 $\sigma$ standard deviation.

In [ ]:
model_test = pyt.tT_model(grain_in,tT_in_test,obs_data)
model_data = model_test.forward()

Finally, if you want to use the mean grain size from the observed data, but your own standard deviation for the $\pm$ grain sizes, you can pass the obs_data to `.tT_model()` and also the keyword variable `std_grain` to `.forward()`:

In [ ]:
model_test = pyt.tT_model(grain_in,tT_in_test,obs_data)
model_data = model_test.forward(std_grain=30)

The model_data variable here is a figure object: 

In [ ]:
type(model_data)

, which you can manipulate as you want to. The following bit of code will save the figure as a png file in your current directory.

In [ ]:
model_data.savefig('dateeU.png')

## A basic example of some lower-level method calls

You may want to adapt the date outputs to suite an alternative plotting style than the one featured in `.forward()`. Below is one example of lower-level method calls that generates one apatite (U-Th)/He date and one zircon (U-Th)/He date from the same tT input array. The comments in the code walk through the process.

First, set up the tT path. This is a simple one that goes from 20 $^\degree$ C to 150 $\degree$ C to 20 $\degree$ C at 500 to 250 to 0 Ma:

In [ ]:
#set up your tT path input array 
tT_in = np.array([[0,20],[250,150],[500,20]])

#create an instance of the tT class with tT_in
#if you want to modify the temp_precision, pass temp_precision = 10 (for example) as the second parameter
tT = pyt.tT_path(tT_in)

#now you can interpolate the path
tT.tT_interpolate()

#and create annealing and reduced time temp arrays, you'll need these for the next two cells below
zirc_anneal,zirc_tT = tT.guenthner_anneal()
ap_anneal,ap_tT = tT.ketcham_anneal()

Here's the apatite cell:

In [ ]:
#set up the input variables
U_ppm = 100
Th_ppm = 50
Sm_ppm = 50
#grain radius in microns
radius = 60
#the total number of nodes used to discretize the grain is 2^log2_nodes + 1, so 257 in this example
#lower numbers run faster, but also give lower precison on the date 
log2_nodes = 8

#creat your instance of an apatite
ap_grain = pyt.apatite(radius,log2_nodes,ap_tT,ap_anneal,U_ppm,Th_ppm,Sm_ppm)

#and calculate the alpha-ejection corrected date using the RDAAM of Flowers et al. (2009)
ap_date = ap_grain.flowers_date()

#let's see the result!
print(ap_date)

And here's the zircon cell:

In [ ]:
#set up the input variables
U_ppm = 1000
Th_ppm = 500
Sm_ppm = 0
#grain radius in microns
radius = 60
#the total number of nodes used to discretize the grain is 2^log2_nodes + 1, so 257 in this example
#lower numbers run faster, but also give lower precison on the date 
log2_nodes = 8

#creat your instance of a zircon
zirc_grain = pyt.zircon(radius,log2_nodes,ap_tT,ap_anneal,U_ppm,Th_ppm,Sm_ppm)

#and calculate the alpha-ejection corrected date using the ZRDAAM of Guenthner et al. (2013)
zirc_date = zirc_grain.guenthner_date()

#let's see the result!
print(zirc_date)